In [ ]:
%pip install 'vanna[snowflake]'

In [2]:
import vanna as vn
import os

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
vn.set_api_key(os.environ['VANNA_API_KEY'])

## Vanna Model
This is the container that will contain the table information and the training queries

In [5]:
vn.set_model('cybersyn-sec-3')

Would you like to create model 'cybersyn-sec-3'? (y/n):  y
What type of database would you like to use? (Snowflake, BigQuery, Postgres, etc.):  Snowflake


## Snowflake Connection
This is a convenience wrapper that just allows you to later do `vn.run_sql` that takes in a SQL string and returns a dataframe.

In [ ]:
vn.connect_to_snowflake(os.environ['SNOWFLAKE_ACCOUNT'], os.environ['SNOWFLAKE_USER'], os.environ['SNOWFLAKE_PASSWORD'], 'CYBERSYN_SEC_COMPANY_FILINGS')

## Get the column information from information schema
We're only using tables columns that are relevant for this dataset.

In [7]:
information_schema = vn.run_sql("""
SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = 'GEOGRAPHY_INDEX' OR TABLE_NAME like 'SEC%'
""")

## Training Plan
This just converts the information schema information into chunks that can be stored in the Vanna model to later be passed to the LLM.

In [8]:
training_plan = vn.get_training_plan_generic(information_schema)
training_plan

Train on Information Schema: CYBERSYN_SEC_COMPANY_FILINGS.CYBERSYN SEC_REPORT_INDEX
Train on Information Schema: CYBERSYN_SEC_COMPANY_FILINGS.CYBERSYN SEC_REPORT_ATTRIBUTES
Train on Information Schema: CYBERSYN_SEC_COMPANY_FILINGS.CYBERSYN SEC_CIK_INDEX
Train on Information Schema: CYBERSYN_SEC_COMPANY_FILINGS.CYBERSYN GEOGRAPHY_INDEX
Train on Information Schema: CYBERSYN_SEC_COMPANY_FILINGS.CYBERSYN SEC_FISCAL_CALENDARS
Train on Information Schema: CYBERSYN_SEC_COMPANY_FILINGS.CYBERSYN SEC_REPORT_TEXT_ATTRIBUTES

## Train
This actually adds the training data to the Vanna model

In [9]:
vn.train(plan=training_plan)

## Get Training Data
Here we're just verifying that the training data made it into the Vanna model for later retrieval

In [10]:
vn.get_training_data()

,id,training_data_type,question,content
0,158-doc,documentation,NaN,The following columns are in the SEC_CIK_INDEX...
1,161-doc,documentation,NaN,The following columns are in the SEC_REPORT_TE...
2,157-doc,documentation,NaN,The following columns are in the SEC_REPORT_AT...
3,160-doc,documentation,NaN,The following columns are in the SEC_FISCAL_CA...
4,159-doc,documentation,NaN,The following columns are in the GEOGRAPHY_IND...
5,156-doc,documentation,NaN,The following columns are in the SEC_REPORT_IN...


## Verify that the Example SQL Runs
Here we're just verifying that the example SQL runs.

In [11]:
vn.run_sql("""
SELECT i.cik, i.company_name, r.period_start_date, r.period_end_date, r.measure_description, TO_NUMERIC(r.value) AS value
FROM cybersyn.sec_cik_index AS i
JOIN cybersyn.sec_report_attributes AS r ON (r.cik = i.cik)
WHERE i.sic_code_description = 'AIR TRANSPORTATION, SCHEDULED'
  AND r.statement = 'Income Statement'
  AND r.period_end_date = '2022-12-31'
  AND r.covered_qtrs = 4
  AND r.metadata IS NULL
  AND r.measure_description IN ('Total operating revenues', 'Total operating revenue');
""")

,CIK,COMPANY_NAME,PERIOD_START_DATE,PERIOD_END_DATE,MEASURE_DESCRIPTION,VALUE
0,0001362468,ALLEGIANT TRAVEL CO,2022-01-01,2022-12-31,Total operating revenues,2301829000
1,0000006201,AMERICAN AIRLINES GROUP INC.,2022-01-01,2022-12-31,Total operating revenues,48971000000
2,0001498710,"SPIRIT AIRLINES, INC.",2022-01-01,2022-12-31,Total operating revenues,5068447000
3,0000100517,"UNITED AIRLINES HOLDINGS, INC.",2022-01-01,2022-12-31,Total operating revenue,44955000000
4,0000027904,"DELTA AIR LINES, INC.",2022-01-01,2022-12-31,Total operating revenue,50582000000
5,0000793733,SKYWEST INC,2022-01-01,2022-12-31,Total operating revenues,3004925000
6,0001670076,"FRONTIER GROUP HOLDINGS, INC.",2022-01-01,2022-12-31,Total operating revenues,3326000000
7,0000092380,SOUTHWEST AIRLINES CO,2022-01-01,2022-12-31,Total operating revenues,23814000000


## Training with Question to SQL Pairs
These 3 SQL statements were provided by the data provider in their documentation. We will use these to train our Vanna model.

In [12]:
vn.train(question="Compare revenue numbers from for 2022 for different airline companies", sql="""
SELECT i.cik, i.company_name, r.period_start_date, r.period_end_date, r.measure_description, TO_NUMERIC(r.value) AS value
FROM cybersyn.sec_cik_index AS i
JOIN cybersyn.sec_report_attributes AS r ON (r.cik = i.cik)
WHERE i.sic_code_description = 'AIR TRANSPORTATION, SCHEDULED'
  AND r.statement = 'Income Statement'
  AND r.period_end_date = '2022-12-31'
  AND r.covered_qtrs = 4
  AND r.metadata IS NULL
  AND r.measure_description IN ('Total operating revenues', 'Total operating revenue');
""")

True

In [13]:
vn.train(question="Measure Chipotle's store count growth over time", sql="""
SELECT i.cik, i.company_name, r.period_end_date, r.measure_description, MAX(TO_NUMBER(r.value)) AS value
FROM cybersyn.sec_cik_index AS i
JOIN cybersyn.sec_report_attributes AS r ON (r.cik = i.cik)
WHERE company_name = 'CHIPOTLE MEXICAN GRILL INC'
AND r.measure_description = 'Number of restaurants'
GROUP BY i.cik, i.company_name, i.cik, r.period_end_date, r.measure_description;
""")

True

In [14]:
vn.train(question="Pull Walmart's fiscal calendar does not align with the calendar year. Pull their quarter start and end dates.", sql="""
SELECT company_name, fiscal_year, fiscal_period, period_start_date, period_end_date
FROM cybersyn.sec_fiscal_calendars
WHERE company_name = 'WALMART INC.'
ORDER BY period_end_date;
""")

True